# Sentiment Analyse von diversen Reviewdaten durch die Nutzung von Wordembeddings und LSTM

## Zielsetzung: 
In diesem Notebook untersuchen wir, ob moderne Methoden wie Word Embedding und LSTMs unsere Genauigkeit weiter verbessern können. Hierfür verwenden wir erneut die Kombination aus McDonald's und IMDB-Bewertungen.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow import keras
from tensorflow.keras import layers
from utils.text_utils import clean_text

2023-07-09 13:11:41.658206: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-09 13:11:41.684419: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-09 13:11:42.073542: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
%pip install kaggle

/usr/bin/zsh: /home/x/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
Note: you may need to restart the kernel to use updated packages.


## Reduzierte Ausführungszeit durch lokale Ausführung
Da nicht jeder über eine leistungsstarke Grafikkarte mit GPU-Unterstützung verfügt, kann die Ausführung von LSTMs zeitaufwändig sein. Eine effektive Alternative zur lokalen Ausführung bietet Google Colab. Dieser Dienst ermöglicht die kostenlose Ausführung des Codes mit GPU-Unterstützung. Obwohl die Authentifizierung bei Diensten wie Kaggle etwas komplexer sein kann, haben wir in den Notebooks Tools integriert, die eine einfache Authentifizierung in Google Colab ermöglichen.

## Authentifizierung bei Kaggle
Navigieren Sie zu https://www.kaggle.com. Gehen Sie dann zu Ihrem [Benutzerprofils](https://www.kaggle.com/me/account) und wählen Sie "API-Token erstellen" aus. Dadurch wird die Datei kaggle.json heruntergeladen, die Ihre API-Zugangsdaten enthalten.

Führen Sie anschließend die nachstehende Zelle aus, um kaggle.json in Ihrer Colab-Laufzeit hochzuladen.

In [3]:
from google.colab import files
    
uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
            name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

ModuleNotFoundError: No module named 'google.colab'

## Herunterladen der Daten

In [ ]:
import os

# Download McDonalds dataset
if not os.path.exists('mcdonalds-store-reviews.zip'):
    print("Downloading McDonalds dataset...")
    !kaggle datasets download -d nelgiriyewithana/mcdonalds-store-reviews
if os.path.exists('mcdonalds-store-reviews.zip'):
    print("Unzipping McDonalds dataset...")
    !unzip -n mcdonalds-store-reviews.zip

# Download IMDB dataset
if not os.path.exists('imdb-dataset-of-50k-movie-reviews.zip'):
    print("Downloading IMDB dataset...")
    !kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
if os.path.exists('imdb-dataset-of-50k-movie-reviews.zip'):
    print("Unzipping IMDB dataset...")
    !unzip -n imdb-dataset-of-50k-movie-reviews.zip

## Prozessierung der Daten
Die Prozessierung wird nicht weiterbeschrieben, weil sie identisch zum letzten Notebook ist. 

In [4]:
df_mc = pd.read_csv('McDonald_s_Reviews.csv', encoding="latin-1")
df_imdb = pd.read_csv('IMDB Dataset.csv')

In [5]:
df_mc = df_mc[['review', 'rating']]
df_imdb = df_imdb[['review', 'sentiment']]

In [6]:
df_mc.head()

,review,rating
0,Why does it look like someone spit on my food?...,1 star
1,It'd McDonalds. It is what it is as far as the...,4 stars
2,Made a mobile order got to the speaker and che...,1 star
3,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,"I repeat my order 3 times in the drive thru, a...",1 star


In [7]:
df_imdb.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
df_mc['review'] = df_mc['review'].apply(clean_text)
df_imdb['review'] = df_imdb['review'].apply(clean_text)    

In [9]:
df_mc.head()

,review,rating
0,look like someone spit food normal transaction...,1 star
1,itd mcdonalds far food atmosphere go staff mak...,4 stars
2,made mobile order got speaker checked line mov...,1 star
3,mc crispy chicken sandwich customer service qu...,5 stars
4,repeat order times drive thru still manage mes...,1 star


In [10]:
df_imdb.head()

,review,sentiment
0,one reviewers mentioned watching oz episode yo...,positive
1,wonderful little production br br filming tech...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically theres family little boy jake thinks...,negative
4,petter matteis love time money visually stunni...,positive


In [ ]:
df_mc = df_mc[df_mc['rating'] != '3 stars']
data_mc = df_mc['review'].to_numpy()
data_imdb = df_imdb['review'].to_numpy()
rating_mapping_imdb = {
    'positive': 1,
    'negative': 0,
}

label_imdb = df_imdb['sentiment'].map(rating_mapping_imdb).to_numpy()
rating_mapping_mc = {
    '1 star': 0,
    '2 stars': 0,
    '4 stars': 1,
    '5 stars': 1
}

label_mc = df_mc['rating'].map(rating_mapping_mc).to_numpy()
data = np.append(data_imdb, data_mc)
label = np.append(label_imdb,label_mc)

train_data, test_data, train_label, test_label = train_test_split(data, label, test_size=0.2, random_state=42)

## Generierung eines Sequence Models aus den Daten
### 1. Text zu Intergers
Um ein Sequenze Model zu erstellen werden die Input Samples als erstes in Sequenzen von Integer Indizes konvertiert, wobei ein Integer ein Wort repräsentiert.  

In [ ]:
max_length = 600
max_tokens = 20000
text_vectorization_sequence = TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
    standardize=None,
)
text_vectorization_sequence.adapt(train_data)
text_vectorization_sequence.adapt(test_data)

int_train_ds = text_vectorization_sequence(train_data)

int_test_ds = text_vectorization_sequence(test_data)

vocabulary = np.array(text_vectorization_sequence.get_vocabulary())

print("Vocabulary size: {}".format(len(vocabulary)))
print("Vocabulary content:\n {}".format(vocabulary[:20]))

## 2. Konvertierung der Integer Sequenzen in Vektor Sequenzen
### 2a. One Hot Encoding
Ein erster Ansatz zur Konvertierung der Integersequenzen in Vektorsequenzen war die Verwendung des One-Hot-Encodings. Dabei würde jede Dimension ein Wort im Vokabular repräsentieren. 

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
    loss="binary_crossentropy",
    metrics=["accuracy"])
model.summary()

In [ ]:
callbacks = [
  keras.callbacks.EarlyStopping(
        monitor="val_loss",  # Metric to monitor
        patience=3,  # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True,  # Restore the weights of the best epoch
    )
]

oh_history = model.fit(int_train_ds, train_label, epochs=10,validation_split=0.2,callbacks=callbacks)

model.evaluate(int_test_ds)

Diese Model trainiert nur sehr langsam oder stürzt bei manchen PCs sogar ab, weil zu wenig RAM vorhanden ist. One-Hot Encoding ist klar die falsche Lösung um diese Daten in ein Vektorsequenzen zu transformieren.

### 2b. Word-Embeddings
Ein wesentliches Problem des One-Hot-Encodings war, dass die Repräsentation keine Zusammenhänge zwischen den Wörtern abbildete, da sie unabhängig voneinander waren. Jedoch sind Wörter nicht unabhängig voneinander. Um diese Abhängigkeiten darzustellen, können Word Embeddings verwendet werden. In diesen reflektieren die geometrischen Beziehungen zwischen zwei Vektoren auch ihre semantischen Beziehungen. 

Keras stellt uns bereits Embedding-Layer zur Verfügung, die wir nutzen können. Diese funktionieren so, dass ihre Gewichtungen (ein internes Dictionary von Token-Vektoren) anfangs zufällig sind. Während des Trainings werden diese Vektoren durch Backpropagation schrittweise angepasst. Die Struktur dieser Layer kann dann von anderen Layern genutzt werden, um die Leistung zu verbessern.

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
 loss="binary_crossentropy",
 metrics=["accuracy"])
model.summary()

## Was ist ein RNNs und LSTMs?
RNNs (Recurrent Neural Networks) sind eine Art von neuronalen Netzwerken, die speziell für die Verarbeitung von sequenziellen Daten entwickelt wurden. Im Gegensatz zu herkömmlichen neuronalen Netzwerken können RNNs Informationen über vergangene Zustände behalten und sie in die Berechnung aktueller Schritte einbeziehen.

Durch ihre Fähigkeit, Kontextinformationen über vergangene Eingaben zu erfassen, können RNNs komplexe Abhängigkeiten in sequenziellen Daten modellieren und lernen. Deshalb können sie auch auf NLP Probleme angewendet werden.

Eine der großen Herausforderungen von RNNs ist das "Vanishing Gradient Problem". Es beschreibt das Verschwinden oder die Explosion des Gradienten während des Backpropagation-Algorithmus, der zum Trainieren des Netzwerks verwendet wird.

Das Vanishing Gradient Problem tritt auf, wenn die Ableitungsfunktionen in den Schichten des Netzwerks Werte kleiner als 1 haben, insbesondere wenn sie nahe bei 0 liegen. In diesem Fall wird der Gradient in jeder Schicht während der Rückpropagierung mit jedem Schritt weiter multipliziert, wodurch er exponentiell kleiner wird. Als Ergebnis erhalten die Gewichte in den ersten Schichten des Netzwerks nur sehr kleine Aktualisierungen, was zu langsamerem oder gar keinem Lernen führen kann. Dies beeinträchtigt insbesondere RNNs, die auf lange Sequenzen angewendet werden.

LSTMs reduzieren dieses Problem durch ihre Architektur. Im Vergleich zu herkömmlichen RNNs verwenden LSTMs einen zusätzlichen Zellzustand, der als "Gedächtnis" fungiert und Informationen über längere Sequenzen hinweg speichert. Der Zellzustand ermöglicht es den LSTMs, wichtige Informationen beizubehalten und irrelevante Informationen zu verwerfen, was dazu beiträgt, das Verschwinden des Gradienten zu verhindern.

In [ ]:
callbacks = [
  keras.callbacks.EarlyStopping(
        monitor="val_loss",  # Metric to monitor
        patience=3,  # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True,  # Restore the weights of the best epoch
    )
]
model.fit(int_train_ds, train_label,validation_split=0.2
, epochs=10,
 callbacks=callbacks)

In [ ]:
model.evaluate(int_test_ds, test_label)

#### Warum sind die Ergbnisse signfikant schlechter als mit dem Bag of Words Model? 
Wir sind noch ein Stück von den Ergebnissen des simpleren Bigram-Modells entfernt. Ein Teil des Grundes dafür ist einfach, dass das Modell etwas weniger Daten betrachtet: Das Bigram-Modell verarbeitete vollständige Bewertungen, während unser Sequenzmodell Sequenzen nach 600 Wörtern abschneidet.
Es gibt hierfür aber noch weitere Gründe. Diese werden im letzen Abschnitt des Notebooks erläutert.

#### Ansehen der Word-Embedding Daten
Um die entstandenen geometrischen Beziehungen zu betrachten, die die semantischen Beziehungen zwischen den Vektoren zeigen, kann das Tool "TensorFlow Projector" verwendet werden. Dazu müssen lediglich die Daten aus dem nächsten Code-Block über den folgenden Link hochgeladen werden: http://projector.tensorflow.org/

In [ ]:
import io
weights = model.get_layer('embedding').get_weights()[0]
vocab = text_vectorization_sequence.get_vocabulary()

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

## Nutzen von vordefinierten Embeddings
Nun nutzen wir die [GloVe](https://github.com/stanfordnlp/GloVe) embeddings von der Stanford NLP Arbeitsgruppe.
Es wird das 100d Model genutzt welches auf Kaggle ([hier](https://www.kaggle.com/datasets/anindya2906/glove6b)) heruntergeladen werden kann. Anschliessend muss dieses in den `embeddings` ordner gelegt werden, sodass `embeddings/glove.6B.100d.txt` zur Verfuegung steht.

In [ ]:
from utils.file_utils import read_embeddings
filepath = './embeddings/glove.6B.100d.txt'
GLOVE_EMBEDDINGS = read_embeddings(filepath)

In [ ]:
test_word = 'hello'
test_vector = GLOVE_EMBEDDINGS[test_word]
print(f"Vektor des Wortes '{test_word}' sieht wie folgt aus:\n\n{test_vector}")

In [ ]:
print(f"Vector shape: {test_vector.shape}")

In [ ]:
embedding_dim = 100
EMBEDDINGS_MATRIX = np.zeros((max_tokens, embedding_dim))

for i, word in np.ndenumerate(vocabulary):
    embedding_vector = GLOVE_EMBEDDINGS.get(word)
    if embedding_vector is not None:
        EMBEDDINGS_MATRIX[i] = embedding_vector

In [ ]:
def create_model(vocab_size, embedding_dim, maxlen, embedding_matrix):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=maxlen, weights=[embedding_matrix], trainable=False),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, dropout=0.5)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        verbose=1,
        patience=5,
        restore_best_weights=True)
    
    callbacks = [early_stopping]    

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy']) 

    return model, callbacks

In [ ]:
model, callbacks = create_model(max_tokens, embedding_dim, max_length, EMBEDDINGS_MATRIX)

In [ ]:
history = model.fit(int_train_ds, train_label, validation_split=0.2, epochs=10, callbacks=callbacks)

In [ ]:
model.evaluate(int_test_ds, test_label)

In [ ]:
from utils.plot_utils import plot_history_metrics
plot_history_metrics(history, ['loss', 'accuracy'])

In [ ]:
from utils.plot_utils import get_classification_report
get_classification_report(model,int_train_ds, train_label)

## Warum performen die Sequence Model Lösungen nicht unbedingt besser als die bag of Gram Lösungen?
Obwohl die Bag-of-Words-Methode (BoW) nicht mehr die aktuellste Technologie im Natural Language Processing (NLP) ist, bietet sie für bestimmte Textklassifikationsaufgaben weiterhin die überlegene Lösung. 

Das Google Keras Team empfiehlt als Heuristik, das Verhältnis zwischen der Anzahl der Samples in den Trainingsdaten und der durchschnittlichen Anzahl von Wörtern pro Sample zu betrachten. Wenn dieses Verhältnis über 1500 liegt, ist ein Sequence Model die geeignetere Lösung, während bei einem Verhältnis unter 1500 ein Bigram die bessere Wahl ist.

In [ ]:
word_counts = []
for item in data: 
    words = item.split()
    word_counts.append(len(words))

word_counts =  np.array(word_counts)


ratio = len(word_counts) / word_counts.mean()
print(ratio)



Bei unseren Daten ist diese Ratio 505.64, deshalb können wir klar sagen, dass Bigrams der bessere Ansatz für die Sentiment Analyse von Reviews dieser Wortanzahl ist. 

Durch das Hinzufügen der IMDB-Trainingsdaten haben wir versucht, ein allgemeingültigeres Modell zu erstellen. Jedoch ist diese Form der Generalisierung mit einem Bigram-Ansatz nur bis zu einer bestimmten Menge an Trainingsdaten effektiv. Wenn wir unsere Trainingsdaten um weitere 500.000 Amazon Reviews erweitern würden und sich der Durchschnitt (Mean) dabei nicht ändern würde, wäre die Verwendung von Sequence Encodings die richtige Wahl.

In [ ]:
withAmazon = (len(word_counts) + 500000 ) /  word_counts.mean()
print(withAmazon)